In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd
from nmme_utils import *

In [2]:
# Set the output path for the data
hcstPath='/data/esplab/shared/model/initialized/nmme/hindcast/'

In [3]:
# Get the model information
nmmemodels_list,_, _,_=initModels()

In [4]:
# Loop over all the Models
for imodel,model in enumerate(nmmemodels_list):
    
    # Get the model, group, variables, and levels from the dictionary     
    dates=model['hcst_dates']
    url_substr=model['url_substr']
    varnames=model['varnames']
    plevstrs=model['levstrs']
    units=model['units']
    model=model['model']
    
    for varname,plevstr,unit in zip(varnames,plevstrs,units):
        
        # Read Data
        
        baseURL=url_substr+varname+'/'
            
        inFname=baseURL+str(datetime.today().toordinal())+'/pop/dods' 
        print(inFname)
        
        # Read Data from IRIDL
        ds=decode_cf(xr.open_dataset(inFname,decode_times=False),'S')
        
        # Convert startdates to datetime64 and extract hindcast years
        ds['S']=ds['S'].astype("datetime64[ns]")
        ds=ds.sel(S=slice(dates[0],dates[1]))

                   
        # Loop over all start times    
        for ic in ds['S'].values:
    
            # Drop the Initial condition date
            ds_out=ds.sel(S=ic).squeeze(drop=True).drop_vars('S')
         
            # Convert coordinate names to something useful
            ds_out=ds_out.rename({'X':'lon','Y':'lat','L':'lead','M':'ens'})
           
            # Set Global Attributes
            ds_out=setattrs(ds_out,units)

            # Create output path if it does not exist
            outPath=hcstPath+varname+'/monthly/full/'+model
            if (not os.path.isdir(outPath)):
                os.makedirs(outPath)
          
            # Write Data to netCDF      
            outFile = outPath+'/'+varname+'_'+model+'_'+pd.to_datetime(ic).strftime('%Y%m')+'.nc'
            print(outFile)
            #ds_out.to_netcdf(outFile,mode='w')                 
    

https://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/NCEP-CFSv2/.HINDCAST/.PENTAD_SAMPLES/.MONTHLY/.tref/738757/pop/dods
/data/esplab/shared/model/initialized/nmme/hindcast/tref/monthly/full/NCEP-CFSv2/tref_NCEP-CFSv2_198201.nc
/data/esplab/shared/model/initialized/nmme/hindcast/tref/monthly/full/NCEP-CFSv2/tref_NCEP-CFSv2_198202.nc
/data/esplab/shared/model/initialized/nmme/hindcast/tref/monthly/full/NCEP-CFSv2/tref_NCEP-CFSv2_198203.nc
/data/esplab/shared/model/initialized/nmme/hindcast/tref/monthly/full/NCEP-CFSv2/tref_NCEP-CFSv2_198204.nc
/data/esplab/shared/model/initialized/nmme/hindcast/tref/monthly/full/NCEP-CFSv2/tref_NCEP-CFSv2_198205.nc
/data/esplab/shared/model/initialized/nmme/hindcast/tref/monthly/full/NCEP-CFSv2/tref_NCEP-CFSv2_198206.nc
/data/esplab/shared/model/initialized/nmme/hindcast/tref/monthly/full/NCEP-CFSv2/tref_NCEP-CFSv2_198207.nc
/data/esplab/shared/model/initialized/nmme/hindcast/tref/monthly/full/NCEP-CFSv2/tref_NCEP-CFSv2_198208.nc
/data/esplab/shared/mo

KeyboardInterrupt: 